# Transformer and question answering

In [ ]:
%pip install -q transformers huggingface_hub
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

Для начала опробуем библиотеку. Попробуем определять тональность текста.

In [ ]:
import transformers
classifier = transformers.pipeline('sentiment-analysis', model="distilbert-base-uncased-finetuned-sst-2-english")

print(classifier("BERT is amazing!"))

In [ ]:
import base64
data = {
    'Losyash': "I just look like a moose, but at heart I'm a butterfly.",
    'Krosh': "The sun is shining - good, not shining - also good, I am my own sun.",
    'Kar Karich': "You can wait your whole life for the right moment and end up never saying something important to each other.",
    'Nyusha': 'If you are not destined to become Miss Universe, what is the point of preening at all?!'
}

outputs = {}# True if positive and False if negative
assert sum(outputs.values()) == 2
print("Well done!")

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-uncased')
model = transformers.AutoModel.from_pretrained('bert-base-uncased')

In [ ]:
tokenizer.tokenize("Let's do tokenization!")

In [ ]:
tokenizer("Let's do tokenization!")

In [ ]:
tokenizer.decode(tokenizer("Let's do tokenization!")["input_ids"])

token_type_ids и attention_mask — это дополнительные значения, которые могут пригодиться при использовании разных моделей. Например, если вы решаете задачу языкового моделирования, то наверняка захотите при помощи attention_mask замаскировать то, что модели надо предсказать (например, вторую половину предложения).

[CLS] и [SEP] — это специальные токены, которые использует BERT. Первый используется для предсказания того, является ли часть B предложением, непосредственно следующим за частью A, а так же используется для обработки глобальной информации. Второй является токеном-разделителем. Токенизатор сам расставил их за нас в данном случае, но иногда приходится самостоятельно проставлять их руками.

Обратите внимание, что каждая обученная модель использует свой токенизатор.

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = torch.tensor([[200, 200, 200]])
sequence2_ids = torch.tensor([[200, 200]])
batched_ids = torch.tensor(
    [
        [200, 200, 200],
        [200, 200, tokenizer.pad_token_id],
    ]
)

print(model(sequence1_ids).logits)
print(model(sequence2_ids).logits)
print(model(batched_ids).logits)

In [ ]:
batched_sequences = [
    "I am a robot and I hate humans",
    "I am a human and i hate robots very much",
]
batched_ids = tokenizer(batched_sequences)["input_ids"]
batched_ids

In [ ]:
!pip install datasets -q

In [ ]:
from datasets import load_dataset

In [ ]:
squad = load_dataset("squad", split="train[:5000]")

In [ ]:
squad = squad.train_test_split(test_size=0.2)

В датасете есть несколько важных полей:

- answers: местоположение первого токена ответа и текст ответа.
- context: исходная информация, из которой модели необходимо извлечь ответ.
 - question: вопрос, на который должна ответить модель.

In [ ]:
sample = squad["train"][4]

То, что находится в поле контекста и вопроса, довольно понятно. Поле ответов немного сложнее, поскольку оно содержит словарь с двумя полями, каждое их которых, в свою очередь, - список. Это формат, который будет использоваться метрикой squad во время оценки. В кастомном случае можно организовывать эти поля как угодно. Поле text содержит ответ на вопрос, а поле answer_start содержит индекс начального символа каждого ответа в контексте.

Во время обучения для каждого семпла есть только один ответ, при валидации же несколько (разной степени конкретности, например)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
inputs = tokenizer(sample["question"], sample["context"])
tokenizer.decode(inputs["input_ids"])

Для дообучения модели нам необходимо предобработать данные. 
У некоторых вопросов контекст слишком длинный, поэтому его нужно сократить до максимальной длины (в нашем случае 386). Для этого нужно установить truncation="only_second".
Затем нужно сопоставить начальную и конечную позиции ответа с исходным контекстом, установив return_offset_mapping=True.
Чтобы потом определить, какая часть смещения соответствует вопросу, а какая — контексту, нужно использовать метод sequence_ids().



In [ ]:
inputs.keys()

In [ ]:
inputs = tokenizer(
        squad["train"][2:6]["question"],
        squad["train"][2:6]["context"],
        max_length=100,
        truncation="only_second",
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

In [ ]:
inputs.keys()

In [ ]:
print(f"The 4 examples gave {len(inputs['input_ids'])} samples.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

Посмотрим на ответы, которые мы должны получить. 

In [ ]:
answers = squad["train"][2:6]["answers"]
answers

In [ ]:
len(inputs["input_ids"][0]), len(inputs["offset_mapping"][0]), inputs["overflow_to_sample_mapping"][0]

Для того, чтобы можно было что-нибудь научить, мы должны каждому семплу поставить в соответствие необходимый ответ. Поэтому придется сделать дополнительный шаг предобработки, который похволит определитб для каждого семпла, содержит ли он ответ или нет, тогда мы должны возвращать особую комбинацию (0, 0)

Чтобы определить, содержится ли ответ в семплах (или был отрезан), и, если необходимо, позиции его токенов, мы сначала должны найти индексы, которые начинают и заканчивают контекст во входных семплах. Для этого мы могли бы использовать идентификаторы типов токенов, но поскольку они не обязательно существуют для всех моделей (например, DistilBERT не требует их), вместо этого мы будем использовать метод Sequence_ids() BatchEncoding, возвращаемый нашим токенизатором.

Получив эти индексы токенов, мы можем взять соответствующие смещения семплов, которые представляют собой кортежи из двух целых чисел, представляющих диапазон символов внутри исходного контекста. Таким образом, мы можем определить, начинается ли фрагмент контекста в этой функции после ответа или заканчивается до начала ответа (в этом случае метка равна (0, 0)). Если это не так, мы выполняем цикл, чтобы найти первый и последний токен ответа:

In [ ]:
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]  # Какому изначальному примеру соответствует семпл
    answer = answers[sample_idx] # Нужный ответ
    start_char = answer["answer_start"][0] # Позиция его начала в исходной последовательности
    end_char = answer["answer_start"][0] + len(answer["text"][0]) 
    sequence_ids = inputs.sequence_ids(i) 

    # Определяем первую и последнюю позицию контекста во входе.
    # None не относится ни к какому семпплу. 0 - вопрос, 1 - ответ
    idx = 0
    
    # определите начало и конец контекста
    # while sequence_ids[idx] != 1 ...
    
    # Если ответ не полностью въодит в контекст, то оставим (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        # your code
    else:
        # В противном случае сохраним первую и последнюю позицию ответа
        idx = context_start
        # найдите первую позицию ответа
        start_positions.append(idx - 1)

        idx = context_end
        # найдите последнюю позицию ответа
        end_positions.append(idx + 1)

start_positions, end_positions

Проверим результат. Сравним значения, полученные с помощью предобработки с таргетами. 

In [ ]:
idx = 3 # поиграйте с индексом - найдите семпл, где ответ влез
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"GT: {answer}, preprocessed: {labeled_answer}")

Теперь мы можем написать функцию предобработки. 

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = # your code (set up tokenizer)

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        # your code - get start and end position 
        # add them to lists

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

Чтобы применить эту функцию ко всему обучающему набору, используем метод Dataset.map() с флагом batched=True. Здесь это необходимо, так как мы меняем длину  датасета за счет разбиения контекста.

In [ ]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

In [ ]:
len(squad["train"]), len(tokenized_squad["train"])

In [ ]:
from transformers import DefaultDataCollator

In [ ]:
data_collator = DefaultDataCollator()

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

Теперь мы можем приступить к обучению модели. У huggingface для этого есть класс trainer, который позволяет максимально легко обучать модели для типичных тасок. Все, о чем нужно позаботиться - это параметры конфига для обучения.

In [ ]:
training_args = TrainingArguments(
    output_dir="qa_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Проверим нашу модель!

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="qa_model")
question_answerer(question=question, context=context)

Задание - попробуйте разные модели.

Но на самом деле мы не ограничены базовым функционалом, поэтому все то же самое мы можем сделать и с помощью torch. Это не слишком отличается от обычного пайплайна обучения. 

In [ ]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

train_dataset = tokenized_squad["train"]
validation_dataset = tokenized_squad["test"]

# Преобразуем датасет в формат torch
train_dataset.set_format("torch")
validation_dataset.set_format("torch")

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    validation_dataset, collate_fn=default_data_collator, batch_size=8
)

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

Настроим акселератор.

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator(fp16=True)
model, optimizer, train_dataloader, eval_dataloader = # prepare all

В этот раз мы не будем считать метрики, так как это потребует достаточно времени. Одюнако при желании вы можете их добавить, чтобы оценить качество модели. 

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        # your code (classic one)
        
    # Eval
    # <your code> - compute the validation loss
    # Save and upload
    # This is more advanced task, use the power of the internet
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)